In [1]:
import json
import Creds
import requests
import pandas as pd
import datetime
import time
import pytz
import csv

In [2]:
last_key = Creds.lastfm_api_key
last_username = Creds.lastfm_username
last_application_name = Creds.lastfm_application_name
last_shared_secret = Creds.lastfm_api_secret
last_registered_to = Creds.lastfm_username

# --- USE VARIAVLES AS NEEDED
# print("Last FM key:", last_key )
# print("Last FM usernamet:", last_username)
# print("Last FM application name:", last_application_name)
# print("Last FM shared secret:", last_shared_secret)
# print("Last FM registered to:", last_registered_to)
print("If all informations show up that means we can go further")

If all informations show up that means we can go further


In [3]:
url =f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist=Cher&api_key={last_key}&format=json"
r = requests.get(url)
# --- CHECK STATUS
#print("status:", r)
data = json.loads(r.text)
#print(json.dumps(data, indent=2))

In [4]:
# --- GENERATE TOKEN IN LAST FM
token_url = f"http://ws.audioscrobbler.com/2.0/?method=auth.gettoken&api_key={last_key}&format=json"

r = requests.get(token_url)
data_token = json.loads(r.text)
#print(json.dumps(data_token, indent=2))
#print(data_token['token'])

In [5]:
# --- GENERATE TOP ARTISTS FROM LAST FM 
top_artist = f"http://ws.audioscrobbler.com/2.0/?method=chart.gettopartists&api_key={last_key}&format=json"

r = requests.get(top_artist)
#print("status:", r)
data_top_artists = json.loads(r.text)
# --- FIRST ARTIST FROM TOP
#print(json.dumps(data_top_artists['artists']['artist'][0], indent=2))

In [6]:
# --- SET PARAMETERS 
limit = 200
page = 1

# --- HOW MANY TRACKS WAS SCROBBLED IN YOUR LAST FM 
recently_music = f"http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=rj&api_key={last_key}&format=json&user={last_username}&limit={limit}&page={page}"
r = requests.get(recently_music)
data_recently_music = json.loads(r.text)

number_tracks = data_recently_music['recenttracks']['@attr']['total']
user_tracks = data_recently_music['recenttracks']['@attr']['user']
total_pages = data_recently_music['recenttracks']['@attr']['totalPages']
per_pages = data_recently_music['recenttracks']['@attr']['perPage']
number_elements = len(data_recently_music['recenttracks']['track'])

print("Number of tracks:", number_tracks)
print("User of tracks:", user_tracks)
print("Total pages:", total_pages)
print("Per page:", per_pages)
#print(number_elements)
# --- VARIABLE ASSIGMENT HOW MANY PAGE ARE BASED ON LIMIT RECORDS ON ONE PAGE
# --- FOR EXAMPLE WHEN WE HAVE 450 TRACKS AND WE SET LIMIT 200 PER PAGE THEN SHOW ME 3 PAGES
# --- CHANGE STRING TO INTEGER
page_limit = int(total_pages)

Number of tracks: 3960
User of tracks: tadeuszshelby
Total pages: 20
Per page: 200


In [7]:
limit = 200

recently_music = f"http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&api_key={last_key}&format=json&user={last_username}&limit={limit}&page=1"
r = requests.get(recently_music)
data_recently_music = json.loads(r.text)

total_pages = data_recently_music['recenttracks']['@attr']['totalPages']

# --- VERY IMPORTANT TOTAL PAGES IS STRING. TO CHECK UNCOMMENT BELOW CODE
#print(type(total_pages))

my_data = []
for page in range(1, int(total_pages) + 1):
       recently_music = f"http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&api_key={last_key}&format=json&user={last_username}&limit={limit}&page={page}"
       print(f"Page {page}: Number of tracks = {number_elements}: Completed")
       r = requests.get(recently_music)
       data_recently_music = json.loads(r.text)
      
       for index, main in enumerate(data_recently_music['recenttracks']['track'], 1):
         data = {
             'artist': main['artist']['#text'],
             'title': main['name'],
             'album': main['album']['#text'],
             'date': main['date']['#text'],
             'url': main['url']
         }
         my_data.append(data)

df = pd.DataFrame(my_data)
df.head()

Page 1: Number of tracks = 200: Completed
Page 2: Number of tracks = 200: Completed
Page 3: Number of tracks = 200: Completed
Page 4: Number of tracks = 200: Completed
Page 5: Number of tracks = 200: Completed
Page 6: Number of tracks = 200: Completed
Page 7: Number of tracks = 200: Completed
Page 8: Number of tracks = 200: Completed
Page 9: Number of tracks = 200: Completed
Page 10: Number of tracks = 200: Completed
Page 11: Number of tracks = 200: Completed
Page 12: Number of tracks = 200: Completed
Page 13: Number of tracks = 200: Completed
Page 14: Number of tracks = 200: Completed
Page 15: Number of tracks = 200: Completed
Page 16: Number of tracks = 200: Completed
Page 17: Number of tracks = 200: Completed
Page 18: Number of tracks = 200: Completed
Page 19: Number of tracks = 200: Completed
Page 20: Number of tracks = 200: Completed


,artist,title,album,date,url
0,Gala,Freed from Desire,Come Into My Life (The Album),"10 Jun 2023, 20:56",https://www.last.fm/music/Gala/_/Freed+from+De...
1,Oasis,Wonderwall,Stop the Clocks,"10 Jun 2023, 18:09",https://www.last.fm/music/Oasis/_/Wonderwall
2,UEFA,UEFA Champions League Anthem,UEFA Champions League Anthem,"10 Jun 2023, 18:07",https://www.last.fm/music/UEFA/_/UEFA+Champion...
3,UEFA,UEFA Champions League Anthem,UEFA Champions League Anthem,"10 Jun 2023, 18:06",https://www.last.fm/music/UEFA/_/UEFA+Champion...
4,Vendredi sur Mer,Mort / Fine,"Beaucoup d'amour, vol. 2","10 Jun 2023, 06:20",https://www.last.fm/music/Vendredi+sur+Mer/_/M...


In [8]:
# --- ALL TRANSFORMATION ON DATE AND TIME COLUMN
from datetime import datetime, timedelta
df['artist'] = df['artist'].astype('string')
df['title'] = df['title'].astype('string')
df['album'] = df['album'].astype('string')
df['url'] = df['url'].astype('string')
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'] + timedelta(hours=2)
# df['date'] = pd.to_datetime(df['date']).dt.date
# df['time'] = pd.to_datetime(df['date']).dt.time
df['new_date'] = [d.date() for d in df['date']]
df['new_time'] = [d.time() for d in df['date']]

# --- ADD NEW COLUMNS OF NUMBER LISTENS 
df['listens'] = 1
df.dtypes

df = pd.DataFrame(df)

# --- RENAME OF COLUMNS IN OUR DATAFRAME DF
df = df.rename (
    columns = 
    {'artist': 'Artist Name',
     'title': 'Song Name',
     'album': 'Album Name',
     'date': 'Datetime of listening',
     'url': 'URL',
     'new_date': 'Date of listening',
     'new_time': 'Time of listening',
     'listens' : 'Number of listens'
    }
)

df.to_csv('Data_Sources/Fact_LastfmData.csv', index=False)

In [9]:
# --- INFORMATION ABOUT USERNAME
user_info = f"http://ws.audioscrobbler.com/2.0/?method=user.getinfo&user=rj&api_key={last_key}&format=json&user={last_username}"

r = requests.get(user_info)
#print("status:", r)

data_user_info = json.loads(r.text)
#print(json.dumps(data_user_info, indent=2))

#print(json.dumps(data_user_info, indent=2))
columns = ['user', 'playcount', 'track_count','artist_account', 'album_count', 'playlist', 'img_account']

info_user = pd.DataFrame(columns = columns)

user = data_user_info['user']['name']
playcount = data_user_info['user']['playcount']
artist_count = data_user_info['user']['artist_count']
track_count = data_user_info['user']['track_count']
album_count = data_user_info['user']['album_count']
playlists = data_user_info['user']['playlists']
img_account = data_user_info['user']['image'][3]['#text']

df_userinfo = pd.DataFrame( 
    {'user': [user],
    'playcount': [playcount],
    'artist_account': [artist_count],
    'track_count': [track_count],
    'album_count': [album_count],
    'playlist': [playlists],
    'img_account': [img_account]
    })

info_user = pd.concat([info_user, df_userinfo])  
#info_user.head()

# --- RENAME OF COLUMNS IN OUR DATAFRAME INFO_USER
info_user = info_user.rename (
    columns = 
    {'user': 'Username',
     'playcount': 'Number of listens',
     'track_count': 'Number of tracks',
     'artist_account': 'Number of artists',
     'album_count': 'Number of albums',
     'playlist': 'Number of playlists',
     'img_account': 'Image account'
    }
)

info_user.to_csv('Data_Sources/Dim_LastfmUsername.csv', index = False)